In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install wandb --upgrade
import wandb
import pathlib
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental.preprocessing import Rescaling
from PIL import Image

Requirement already up-to-date: wandb in /usr/local/lib/python3.7/dist-packages (0.10.25)


In [ ]:
################################################################
# Preparing training (without augmentation) and validation set 
################################################################
# Preparing training and validation sets without augmentation
# Loading data from directory
data_dir = pathlib.Path('/content/drive/MyDrive/inaturalist_12K/train') # Set path to the right directory
train_data = tf.keras.preprocessing.image_dataset_from_directory(
                      directory = data_dir,
                      labels = 'inferred',  
                      label_mode = 'categorical',
                      color_mode = 'rgb',
                      batch_size = 32,
                      image_size = (256, 256),
                      shuffle = True,
                      seed = 17,
                      validation_split = 0.2,
                      subset = 'training')

val_data = tf.keras.preprocessing.image_dataset_from_directory(
                      directory = data_dir,
                      labels = 'inferred',  
                      label_mode = 'categorical',
                      color_mode = 'rgb',
                      batch_size = 32,
                      image_size = (256, 256),
                      shuffle = True,
                      seed = 17,
                      validation_split = 0.2,
                      subset = 'validation')

# Retaining 25 percent of train and validation data and discarding the rest
len_train, len_val = len(train_data), len(val_data)
train_data = train_data.take(int(0.25*len_train))
val_data = val_data.take(int(0.25*len_val))

Found 10000 files belonging to 10 classes.
Using 8000 files for training.
Found 10000 files belonging to 10 classes.
Using 2000 files for validation.


In [ ]:
train_data

<TakeDataset shapes: ((None, 256, 256, 3), (None, 10)), types: (tf.float32, tf.float32)>

In [ ]:
################################################################
# Preparing training set with augmentation 
################################################################
train_data_augmenter = ImageDataGenerator(
                            rescale = None,
                            rotation_range = 20,
                            width_shift_range = 0.2,
                            height_shift_range = 0.2,
                            brightness_range = [0.2, 1.5],
                            shear_range = 0.2,
                            zoom_range = 0.2,
                            horizontal_flip=True,
                            data_format = 'channels_last',
                            validation_split = 0.2)        #Specifying parameters for augmentation of training data

val_data_augmenter = ImageDataGenerator(validation_split = 0.2) #No augmentation of validation data

train_aug_gen = train_data_augmenter.flow_from_directory(data_dir, shuffle = True, \
                                                         seed = 17, subset = 'training')
val_aug_gen = val_data_augmenter.flow_from_directory(data_dir, shuffle = True, \
                                                     seed = 17, subset = 'validation')

train_aug_data = tf.data.Dataset.from_generator(
                    lambda: train_aug_gen,
                    output_types = (tf.float32, tf.float32),
                    output_shapes = ([None, 256, 256, 3], [None, 10]))

val_aug_data = tf.data.Dataset.from_generator(
                  lambda: val_aug_gen,
                  output_types = (tf.float32, tf.float32),
                  output_shapes = ([None, 256, 256, 3], [None, 10]))

# Retaining 25 percent of train and validation data and discarding the rest
train_aug_data = train_aug_data.take(int(0.25*len_train))
val_aug_data = val_aug_data.take(int(0.25*len_val))

Found 8001 images belonging to 10 classes.
Found 1999 images belonging to 10 classes.


In [ ]:
train_aug_data

<TakeDataset shapes: ((None, 256, 256, 3), (None, 10)), types: (tf.float32, tf.float32)>

In [ ]:
###############################################
# Listing the hyperparameters in wandb config 
###############################################
sweep_config = {'name': 'random-test-sweep', 'method': 'random'}
sweep_config['metric'] = {'name': 'val_acc', 'goal': 'maximize'}
parameters_dict = {
                   'first_layer_filters': {'values': [32, 64]},
                   'filter_org': {'values': [0.5, 1]}, # Halving, same, doubling in subsequent layers
                   'data_aug': {'values': [True]},
                   'batch_norm': {'values': [True]}, 
                   'dropout': {'values': [0.0, 0.2, 0.3]},
                   'kernel_size': {'values': [3]},
                   'dense_size': {'values': [32, 64, 128]},
                   'activation': {'values': ['relu']},
                   'num_epochs': {'values': [50]}, 
                   'optimizer': {'values': ['adam']},
                   'conv_layers': {'values': [5]}
                  }
sweep_config['parameters'] = parameters_dict


In [ ]:
#####################################
# Defining the train function
#####################################
def CNN_train(config=sweep_config):
    with wandb.init(config=config):
        config = wandb.init().config
        wandb.run.name = 'firstLayerFilters_{}_filterOrg_{}_dataAug_{}_batchNorm_{}_dropout_{}_kerSize_{}_denseSize_{}'\
                         .format(config.first_layer_filters, config.filter_org, config.data_aug, config.batch_norm, config.dropout, config.kernel_size, config.dense_size)
        
        ###########################################
        # Initializing the model architecture
        ###########################################
        inputs = tf.keras.Input(shape = (256, 256, 3))
        x = Rescaling(scale = 1.0/255)(inputs)
        filter_sizes = [int(config.first_layer_filters*(config.filter_org**layer_num)) for layer_num in range(config.conv_layers)]
        ker_size = config.kernel_size

        # Apply some convolution and pooling layers
        for layer_num in range(config.conv_layers):
            x = layers.Conv2D(filters = filter_sizes[layer_num], kernel_size = (ker_size, ker_size))(x)
            if config.batch_norm:
                x = layers.BatchNormalization(axis = -1)(x)
            x = layers.Activation(config.activation)(x)
            x = layers.MaxPooling2D(pool_size = (2, 2))(x)            
                
        # Dense Layer
        x = layers.Flatten()(x)
        x = layers.Dense(config.dense_size)(x)
        if config.batch_norm:
            x = layers.BatchNormalization(axis = -1)(x)
        x = layers.Activation(config.activation)(x)
        if config.dropout > 0:
            x = layers.Dropout(rate = config.dropout)(x)        

        # Output Layer
        outputs = layers.Dense(10, activation ='softmax')(x)
        model = tf.keras.Model(inputs = inputs, outputs = outputs)
        #print(model.summary())
        ####################################
        # Training and evaluating the model
        ####################################
        # Using training data with or without augmentation
        sweep_train_data = train_aug_data if config.data_aug else train_data
        sweep_val_data = val_aug_data if config.data_aug else val_data # In any case, validation data is not augmented
        
        model.compile(optimizer=config.optimizer,
                      loss = tf.keras.losses.CategoricalCrossentropy(name = 'loss'),
                      metrics = [tf.keras.metrics.CategoricalAccuracy(name = 'acc')])
        
        # Fitting the model and logging metrics (train_loss, train_acc, val_loss, val_acc) after every epoch
        model_hist = model.fit(sweep_train_data, epochs = config.num_epochs,
                               validation_data = sweep_val_data, 
                               callbacks = [tf.keras.callbacks.EarlyStopping(monitor = 'val_acc', patience = 5),
                                            wandb.keras.WandbCallback()])

In [ ]:
#################################
# Setting up wandb sweeps
#################################
sweep_id = wandb.sweep(sweep_config, project = 'DL-Assignment2-PartA-5April-2')
wandb.agent(sweep_id, function=CNN_train)

Create sweep with ID: firnynx9
Sweep URL: https://wandb.ai/cs6910-team/DL-Assignment2-PartA-5April-2/sweeps/firnynx9


wandb: Agent Starting Run: 75xanm09 with config:
wandb: 	activation: relu
wandb: 	batch_norm: True
wandb: 	conv_layers: 5
wandb: 	data_aug: True
wandb: 	dense_size: 64
wandb: 	dropout: 0.3
wandb: 	filter_org: 1
wandb: 	first_layer_filters: 32
wandb: 	kernel_size: 3
wandb: 	num_epochs: 50
wandb: 	optimizer: adam
wandb: Currently logged in as: cs6910-team (use `wandb login --relogin` to force relogin)


Epoch 1/50
62/62 [==============================] - 104s 1s/step - loss: 2.6069 - acc: 0.1224 - val_loss: 2.3555 - val_acc: 0.0958
Epoch 2/50
62/62 [==============================] - 71s 1s/step - loss: 2.3364 - acc: 0.1605 - val_loss: 2.4004 - val_acc: 0.1104
Epoch 3/50
62/62 [==============================] - 71s 1s/step - loss: 2.2678 - acc: 0.1966 - val_loss: 2.4575 - val_acc: 0.1250
Epoch 4/50
62/62 [==============================] - 72s 1s/step - loss: 2.2523 - acc: 0.1784 - val_loss: 2.3998 - val_acc: 0.0979
Epoch 5/50
62/62 [==============================] - 70s 1s/step - loss: 2.1841 - acc: 0.2064 - val_loss: 2.2999 - val_acc: 0.1210
Epoch 6/50
62/62 [==============================] - 71s 1s/step - loss: 2.2008 - acc: 0.2021 - val_loss: 2.3175 - val_acc: 0.1354
Epoch 7/50
62/62 [==============================] - 71s 1s/step - loss: 2.1811 - acc: 0.2222 - val_loss: 2.1949 - val_acc: 0.1896
Epoch 8/50
62/62 [==============================] - 70s 1s/step - loss: 2.1688 - acc: 0.2

wandb: Sweep Agent: Waiting for job.
wandb: Job received.
wandb: Agent Starting Run: 5dd724e3 with config:
wandb: 	activation: relu
wandb: 	batch_norm: True
wandb: 	conv_layers: 5
wandb: 	data_aug: True
wandb: 	dense_size: 64
wandb: 	dropout: 0
wandb: 	filter_org: 0.5
wandb: 	first_layer_filters: 64
wandb: 	kernel_size: 3
wandb: 	num_epochs: 50
wandb: 	optimizer: adam


Epoch 1/50
62/62 [==============================] - 73s 1s/step - loss: 2.4666 - acc: 0.1136 - val_loss: 2.3192 - val_acc: 0.1063
Epoch 2/50
62/62 [==============================] - 70s 1s/step - loss: 2.2919 - acc: 0.1391 - val_loss: 2.4047 - val_acc: 0.0667
Epoch 3/50
62/62 [==============================] - 71s 1s/step - loss: 2.2272 - acc: 0.1767 - val_loss: 2.3916 - val_acc: 0.1058
Epoch 4/50
62/62 [==============================] - 74s 1s/step - loss: 2.1970 - acc: 0.1920 - val_loss: 2.4862 - val_acc: 0.0750
Epoch 5/50
62/62 [==============================] - 74s 1s/step - loss: 2.2016 - acc: 0.1876 - val_loss: 2.5452 - val_acc: 0.0771
Epoch 6/50
62/62 [==============================] - 73s 1s/step - loss: 2.1760 - acc: 0.1954 - val_loss: 2.4215 - val_acc: 0.1250
Epoch 7/50
62/62 [==============================] - 71s 1s/step - loss: 2.1532 - acc: 0.2036 - val_loss: 2.3196 - val_acc: 0.1404
Epoch 8/50
62/62 [==============================] - 73s 1s/step - loss: 2.1276 - acc: 0.21

...

wandb: Agent Starting Run: 3vvzrj50 with config:
wandb: 	activation: relu
wandb: 	batch_norm: True
wandb: 	conv_layers: 5
wandb: 	data_aug: True
wandb: 	dense_size: 128
wandb: 	dropout: 0.3
wandb: 	filter_org: 0.5
wandb: 	first_layer_filters: 64
wandb: 	kernel_size: 3
wandb: 	num_epochs: 50
wandb: 	optimizer: adam


Epoch 1/50
62/62 [==============================] - 74s 1s/step - loss: 2.7035 - acc: 0.1110 - val_loss: 2.3119 - val_acc: 0.1000
Epoch 2/50
62/62 [==============================] - 72s 1s/step - loss: 2.4500 - acc: 0.1290 - val_loss: 2.3546 - val_acc: 0.0771
Epoch 3/50
62/62 [==============================] - 72s 1s/step - loss: 2.3408 - acc: 0.1500 - val_loss: 2.3975 - val_acc: 0.1274
Epoch 4/50
62/62 [==============================] - 74s 1s/step - loss: 2.3413 - acc: 0.1634 - val_loss: 2.3960 - val_acc: 0.1042
Epoch 5/50
62/62 [==============================] - 73s 1s/step - loss: 2.2675 - acc: 0.1921 - val_loss: 2.3840 - val_acc: 0.0958
Epoch 6/50
62/62 [==============================] - 72s 1s/step - loss: 2.2526 - acc: 0.1805 - val_loss: 2.4192 - val_acc: 0.1167
Epoch 7/50
62/62 [==============================] - 72s 1s/step - loss: 2.2256 - acc: 0.1954 - val_loss: 2.3030 - val_acc: 0.1490
Epoch 8/50
62/62 [==============================] - 73s 1s/step - loss: 2.2145 - acc: 0.17

wandb: Agent Starting Run: 3sd2cunk with config:
wandb: 	activation: relu
wandb: 	batch_norm: True
wandb: 	conv_layers: 5
wandb: 	data_aug: True
wandb: 	dense_size: 32
wandb: 	dropout: 0.2
wandb: 	filter_org: 1
wandb: 	first_layer_filters: 32
wandb: 	kernel_size: 3
wandb: 	num_epochs: 50
wandb: 	optimizer: adam


Epoch 1/50
     22/Unknown - 24s 1s/step - loss: 2.6613 - acc: 0.1099

wandb: Ctrl + C detected. Stopping sweep.
